### device model

First model :
    - XDGBoost : overfit (like crazy) 
        result : accuracy = 0.93
            

==> trying RandomForest instead
       better, less overfit
       
- increasing the nb folds during hyperpparameters
- removing the features that should not make sense:
    - attribute1
    - min_any and std_any
    
==> 'reasonable' model, 93% accuracy, but much less overfit

 - removing suspicious positives (see dataset exploration)


# going through features...

 - attribute1 : removed
 - attribute2 :
   - useful as raw feature (max & mean)
   - removing it lower perfs : accuracy : 92.4 --> 92.4... so no use
   - adding dt_attribute2 ==> 92.4-> 92.77
   - addinf dt2_attribute2 ==> 92.7 -> 93.2 (93.5 with calibration)
 